In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
import numpy as np
import os
from IPython.display import Image, display
from common.data_model import DataModel
from curated.orders_lean.models.transformer import OrderLeanTransformer, PaymentTransformer


In [3]:
partition_dates = [ "2016-09-04", "2016-09-11"]

In [4]:
# Args
ZONE = "raw"


# load
DataModel.set_mode(local=True)
orders = DataModel.read_partitioned_dataframe(zone=ZONE, dataset="olist_orders_dataset", partition_dates = partition_dates)
items = DataModel.read_partitioned_dataframe(zone=ZONE, dataset="olist_order_items_dataset", partition_dates = partition_dates)
payments = DataModel.read_partitioned_dataframe(zone=ZONE, dataset="olist_order_payments_dataset", partition_dates = partition_dates)
seller = DataModel.read_dataframe(zone=ZONE, dataset="olist_sellers_dataset.parquet")
customer = DataModel.read_dataframe(zone=ZONE, dataset="olist_customers_dataset.parquet")


In [6]:
PaymentTransformer.transform_payment(payments)

,order_id,payment_total_credit_card,number_of_payments_credit_card,number_of_installments_credit_card,payment_total_sum,main_payment_type
0,2e7a8482f6fb09756ca50c10d7bfc047,136.23,1,1,136.23,credit_card
1,e5fa5a7210941f7d56d0208e4e071d35,75.06,1,3,75.06,credit_card


In [10]:
# transform
order_lean = OrderLeanTransformer.curate_orders_lean(orders, payments, items, seller, customer)


In [14]:

# save
DataModel.write_partitioned_dataframe(order_lean, zone="transient", dataset="orders_lean", partition_column="purchase_date")

(98163, 35)


,order_id,seller_id,customer_unique_id,order_status,order_created_date,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,...,total_freight,n_items,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state,residual,is_payment_madeup
0,e481f51cbdc54678b7cc49136f2d6af7,3504c0cb71d7fa48d967e0e4c94d59d9,7c396fd4830fd04220f754e42b4e5bff,delivered,2017-10-02,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,...,8.72,1,3149,sao paulo,SP,9350,maua,SP,0.0,False
1,53cdb2fc8bc7dce0b6741e2150273451,289cdb325fb7e7f891c38608bf9e0962,af07308b275d755c9edb36a90c618231,delivered,2018-07-24,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,...,22.76,1,47813,barreiras,BA,31570,belo horizonte,SP,0.0,False
2,47770eb9100c2d0c44946d9cf07ec65d,4869f7a5dfa277a7dca6462dcf3b52b2,3a653a41f6f9fc3d2a113cf8398680e8,delivered,2018-08-08,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,...,19.22,1,75265,vianopolis,GO,14840,guariba,SP,0.0,False
3,949d5b44dbf5de918fe9c16f97b45f8a,66922902710d126a0e7d26b0e3805106,7c142cf63193a1473d2e66489a9ae977,delivered,2017-11-18,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,...,27.20,1,59296,sao goncalo do amarante,RN,31842,belo horizonte,MG,0.0,False
4,ad21c59c0840e6cb83a9ceb5573f8159,2c9e548be18521d1c43cde1c582c6de8,72632f0f9dd73dfee390c9b22eb56dd6,delivered,2018-02-13,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,...,8.72,1,9195,santo andre,SP,8752,mogi das cruzes,SP,0.0,False
